### 2.3.1 File Channel Transfer
#### 一. Channel之间的读写
1. NIO中, 如果有2个Channel, 其中一个Channel是`FileChannel`, 则数据可以直接从一个Channel读入/写出到另一个Channel中  
2. `FileChannel`实现了`transferFrom`和`transferTo`

#### 二. transferFrom()
1. 如下打开两个文件channel,把数据从一个文件写入到另一个文件.
    * 下面的position和count用来描述源文件.
        * position: 从toChannel的哪个位置开始写
        * count: 从fromChannel中读多少数据写入 
        
 <img src="img/transforfrom.png" width="70%">
    
    
2. 如果FileChannel的transferFrom是SocketChannel,那么当调用transferFrom()方法时, 会立刻把数据从socket的缓冲区中读取出来; 即便读取的大小没到count指定的大小, 且后续仍会有数据从socket中来, 也不能再次写入到FileChannel中

#### 三. transferTo()
1. 同上述功能一样, 将数据从fromChannel拷贝到toChannel
    * position: 从fromChannel的那个位置开始读
    * count: 从fromChannel中读多少数据
2. 如果fromChannel是SocketChannel, 一样会存在transferFrom的问题: 立刻读取, 且后续即使有数据到来也不能继续读取
    <img src="img/transforto.png" width="70%">

### 2.3.2 File Channel基本操作

#### 1. 打开FileCHannel
* (1) FIleChannel的获取方式有2种:
    * 输入流, 输出流
    * 从RandomAccessFile中获取
    ```java
    RandomAccessFile aFile = new RandomAccessFile("data/nio-data.txt", "rw");
    FileChannel inChannel = aFile.getChannel();
    ```    
    
#### 2. read data from FileChanel
* `FileChannel.read()`方法, 返回的int表示读了多少字节的数据到buffer中
* 如果`FileChannel.read()`方法返回-1, 则表示已经读到了文件末尾
```java
ByteBuffer buf = ByteBuffer.allocate(48);
int bytesRead = inChannel.read(buf);
```

#### 3. Writing Data to a FileChannel
* ` FileChannel.write()`方法并不是一次性将buffer中的所有字节写入channel, 而是每次写入一点. 因此, ` FileChannel.write()`方法应该写在循环内部, 判断条件是`buffer.hasRemaining`
```java
buf.flip();
while(buf.hasRemaining()) {
    channel.write(buf);
}
```

#### 4. Closing a FileChannel与FileChannel Size
* 使用`channel.close()`关闭Channel
* 使用`channel.truncate(1024);`

#### 5. FileChannel Force
* 使用`FileChannel.force()`方法, 强制将channel中未写入磁盘的数据写入磁盘. 
* 为什么使用这个方法?  
   因为操作系统的设计, 往往会用一个缓存缓冲写入磁盘的数据, 调用这个方法可以强制将魂村中的数据写入磁盘, 来保证数据完整性
   
#### 6. 另一种拷贝文件方法(不如transfer高效)
```java
String dir = "/Users/lj/IdeaProjects/JavaFeatures/Java NetWork/";
RandomAccessFile fromFile = new RandomAccessFile(dir+"data/from.txt","r");
FileChannel fromChannel = fromFile.getChannel();
RandomAccessFile toFile = new RandomAccessFile(dir+"data/to.txt", "rw");
FileChannel toChannel = toFile.getChannel();

long count = fromChannel.size();
System.out.println("count is:"+count);

ByteBuffer buffer = ByteBuffer.allocate(32);  // 申请一块内存缓冲
int hasRead ;
while((hasRead=fromChannel.read(buffer))!=-1){
    buffer.flip();  // 准备进入读模式
    while(buffer.hasRemaining()){
        toChannel.write(buffer);
    }
    buffer.clear(); // 准备进入写模式
}

toChannel.force(true); // 强制刷新channel中未写出的数据到磁盘
```